<a href="https://colab.research.google.com/github/moustafa123/teacher-assistant-chatbot/blob/main/TeacherAssistantBloom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# CSI 5180 Dataset Generated
dataset = pd.read_csv("VA-Questions-Answers.csv")

In [ ]:
dataset.dropna()
dataset.shape

In [ ]:
dataset.shape

In [ ]:
# Print Dataset Head
dataset.head()

In [ ]:
# Build the prompt that will be passed to the model
def buildprompts(data):
    prompt = {}
    prompt['text'] = "Given the question delimited by triple backticks ```{" + data['question'] + "}```, what is the answer? Answer: {" + data['answer'] + "}"
    return prompt

In [ ]:
dataset['prompt'] = dataset.apply(buildprompts, axis=1)

In [ ]:
print(dataset['prompt'])

In [ ]:
# Insert the prompt dataset in a json file
result = dataset['prompt'].to_list()
with open('/content/sample_data/prompts.json', 'w') as outfile:
    json.dump(result, outfile, ensure_ascii=False)

In [ ]:
!pip install transformers
!pip install accelerate -U
!pip install datasets

In [ ]:
import pandas as pd
import torch
import json
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from accelerate import Accelerator

In [ ]:
# Get the tokenizer and model from bloomz-560m
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloomz-560m")
model = BloomForCausalLM.from_pretrained("bigscience/bloomz-560m").to("cuda")

In [ ]:
# Load json dataset
dataset = load_dataset("json", data_files="/content/sample_data/prompts.json")

In [ ]:
dataset

In [ ]:
# Function to prepare the training data
def prepare_train_data(data):
    # prompt + completion
    #text_input = data['prompt'] + ' ' + data['completion']
    text_input = data['text']
    # tokenize the input (prompt + completion) text
    tokenized_input = tokenizer(text_input, return_tensors='pt', padding=True)
    # generative models: labels are the same as the input
    tokenized_input['labels'] = tokenized_input['input_ids']
    return tokenized_input

In [ ]:
train_dataset = dataset['train'].map(prepare_train_data, batched=True, remove_columns=["text"])

In [ ]:
train_dataset

In [ ]:
!pip install accelerate -U
!pip install transformers==4.30

In [ ]:
import pandas as pd
import torch
import json
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from accelerate import Accelerator

In [ ]:
# Create the Training Arguments
training_arguments = TrainingArguments(
    'LegalQA-bloom-560m-dev',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,
    optim="adafactor",
    gradient_accumulation_steps=4,
    gradient_checkpointing=True
)

In [ ]:
# Create the trainer based on the Training Arguments
trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset
)

In [ ]:
# Train the model on the dataset
trainer.train()

In [ ]:
# Save a local copy of the model to be used in the future
trainer.save_model()

In [ ]:
import torch
from transformers import pipeline
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments, Trainer

In [ ]:
# Test the model
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloomz-560m")
model = BloomForCausalLM.from_pretrained("LegalQA-bloom-560m",low_cpu_mem_usage=True).to("cpu")
prompt = 'Given the question delimited by triple backticks ```{What is NLP?}```, what is the answer? Answer:'
generator = pipeline('text-generation', model=model, tokenizer=tokenizer,do_sample=False)
result = generator(prompt, max_length=128)
print(result)